# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [1]:
%%file count_letters.py
from collections import Counter
def count_letters(file):
    with open(file, encoding='windows-1251') as fp:
        text = fp.read().lower()
    return Counter(text)

Overwriting count_letters.py


In [2]:
from count_letters import count_letters

In [3]:
%%time

count_letters('data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')
count_letters('data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt')

CPU times: user 139 ms, sys: 5.69 ms, total: 145 ms
Wall time: 144 ms


Counter({'с': 50084,
         'п': 25652,
         'а': 73555,
         'и': 62030,
         'б': 16016,
         'о': 106740,
         ',': 26973,
         ' ': 182305,
         'ч': 16492,
         'т': 59813,
         'к': 30802,
         'л': 42328,
         'н': 60920,
         'г': 16174,
         'у': 27309,
         'в': 43700,
         'е': 80972,
         'й': 9747,
         'э': 3203,
         'р': 39784,
         'b': 25,
         'o': 104,
         'k': 16,
         's': 96,
         'c': 42,
         'a': 98,
         'f': 23,
         'e': 162,
         '.': 9864,
         'n': 114,
         't': 98,
         ':': 984,
         'h': 48,
         'p': 29,
         '/': 22,
         '\n': 8583,
         'u': 86,
         'r': 76,
         'd': 38,
         'v': 65,
         'i': 235,
         'y': 5,
         '_': 8,
         '-': 3558,
         '1': 384,
         '0': 110,
         '9': 100,
         '6': 271,
         'm': 54,
         'l': 46,
         'ж': 10552,
     

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [4]:
import multiprocessing as mp

In [5]:
%time
if __name__ == '__main__':
    files = ['data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt',
       'data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt']
    with mp.Pool(processes=len(files)) as pool:
        counters = pool.map(count_letters, files)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.86 µs


In [6]:
type(counters)

list

In [7]:
counters[1]

Counter({'с': 50084,
         'п': 25652,
         'а': 73555,
         'и': 62030,
         'б': 16016,
         'о': 106740,
         ',': 26973,
         ' ': 182305,
         'ч': 16492,
         'т': 59813,
         'к': 30802,
         'л': 42328,
         'н': 60920,
         'г': 16174,
         'у': 27309,
         'в': 43700,
         'е': 80972,
         'й': 9747,
         'э': 3203,
         'р': 39784,
         'b': 25,
         'o': 104,
         'k': 16,
         's': 96,
         'c': 42,
         'a': 98,
         'f': 23,
         'e': 162,
         '.': 9864,
         'n': 114,
         't': 98,
         ':': 984,
         'h': 48,
         'p': 29,
         '/': 22,
         '\n': 8583,
         'u': 86,
         'r': 76,
         'd': 38,
         'v': 65,
         'i': 235,
         'y': 5,
         '_': 8,
         '-': 3558,
         '1': 384,
         '0': 110,
         '9': 100,
         '6': 271,
         'm': 54,
         'l': 46,
         'ж': 10552,
     

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

Фрагмент одного из файлов, которые должны получиться в результате:
```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [18]:
import csv
#ast.eval
count = 8
flags = [0 for i in range(8)]
with open('data/recipes_full.csv') as r:  
    reader = csv.DictReader(r)
    for i,row in enumerate(reader):
        with open(f'data/splitfiles/file{i%count}.csv','a+', newline='') as w:
            if(flags[i%count] == 0):
                flags[i%count] = 1
                csv.writer(w,delimiter =';').writerow(['id','tag','n_steps'])
            for tag in eval(row['tags']):
                csv.writer(w,delimiter =';').writerow([row['id'],tag,row['n_steps']])

2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [7]:
import csv
def mean_f(name):
    tag_step = {}
    tag_step_cnt = {}
    with open(name) as csvfile:
        reader = csv.DictReader(csvfile,delimiter= ';')
        for row in reader:
            if row['tag'] not in tag_step:
                tag_step[row['tag']] = 0
                tag_step_cnt[row['tag']] = 0
            tag_step[row['tag']] += int(row['n_steps'])
            tag_step_cnt[row['tag']] += 1
    for tag in tag_step:
        if tag_step_cnt[tag] !=0:
            tag_step[tag] = tag_step[tag]/tag_step_cnt[tag]
    return tag_step

In [8]:
mean_f('data/splitfiles/file1.csv')

{'brunch': 6.7841180299414185,
 'ham-and-bean-soup': 3.509492273730684,
 'colombian': 3.5896309314586996,
 'savory-pies': 4.27456647398844,
 'refrigerator': 4.7487889273356405,
 'australian': 4.255,
 'served-cold': 4.931677018633541,
 'spaghetti': 4.106,
 'dips-summer': 3.487078405606658,
 'chutneys': 3.586818181818182,
 'mardi-gras-carnival': 3.604294478527607,
 'small-appliance': 5.185928143712575,
 'holiday-event': 8.222388506435198,
 'desserts-easy': 3.489786654561961,
 'high-protein': 4.927578402845135,
 'beginner-cook': 6.361737054171019,
 'bacon': 4.159108678655199,
 'dairy-free': 3.5598946906537954,
 '15-minutes-or-less': 5.00625488663018,
 '30-minutes-or-less': 7.552597119161938,
 'lime': 3.686517783291977,
 'czech': 3.569113012156686,
 'iranian-persian': 3.572208228379513,
 'saudi-arabian': 3.5502202643171805,
 'valentines-day': 4.5369127516778525,
 'zucchini': 3.5191208791208792,
 'roast-beef': 4.028655710071639,
 'caribbean': 3.960078277886497,
 'bean-soup': 3.5060816681146

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [9]:
import csv
def mean_all_files():
    tag_step = {}
    tag_step_cnt = {}
    files = [f'data/splitfiles/file{i}.csv' for i in range(8)]
    for name in files:
        with open(name) as csvfile:
            reader = csv.DictReader(csvfile,delimiter= ';')
            for row in reader:
                if row['tag'] not in tag_step:
                    tag_step[row['tag']] = 0
                    tag_step_cnt[row['tag']] = 0
                tag_step[row['tag']] += int(row['n_steps'])
                tag_step_cnt[row['tag']] += 1
    for tag in tag_step:
        if tag_step_cnt[tag] !=0:
            tag_step[tag] = tag_step[tag]/tag_step_cnt[tag]
    return tag_step

In [10]:
%%time 
mean_all_files()

CPU times: user 27.8 s, sys: 238 ms, total: 28 s
Wall time: 28 s


{'mexican': 5.302344316442439,
 'healthy-2': 6.384162244806188,
 'orange-roughy': 3.513425052701653,
 'chicken-thighs-legs': 4.145581465931509,
 'freezer': 4.033042234819468,
 'whitefish': 3.514734127201888,
 'pork-sausage': 4.256068444090729,
 'filipino': 3.575355140695586,
 'for-large-groups': 7.292883853009813,
 'pasta-salad': 3.5048206710374084,
 'rosh-hashana': 3.6900091857135138,
 'cambodian': 3.512109589041096,
 'pasta': 6.583805408865419,
 'fruit': 7.310792710151398,
 'cabbage': 3.5122165004930426,
 'grains': 5.206610190710679,
 'equipment': 8.574089581965914,
 'lime': 3.674473934651581,
 'low-sodium': 7.3863798933541425,
 'bass': 3.5415725384318697,
 'meatballs': 3.541812330059375,
 'veal': 3.6805035541809104,
 'prepared-potatoes': 3.5132198880328143,
 'oaxacan': 3.5224962918200298,
 'collard-greens': 3.5319676193622995,
 'pheasant': 3.520208171852508,
 'fudge': 3.726083488433927,
 'micro-melanesia': 3.5141301957334505,
 'cajun': 3.910293213040804,
 'breakfast-eggs': 3.5108534

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [11]:
%%file mean_f.py
from csv import DictReader
def mean_f(name):
    tag_step = {}
    tag_step_cnt = {}
    with open(name) as csvfile:
        reader = DictReader(csvfile,delimiter = ';')
        for row in reader:
            if row['tag'] not in tag_step:
                tag_step[row['tag']] = 0
                tag_step_cnt[row['tag']] = 0
            tag_step[row['tag']] += int(row['n_steps'])
            tag_step_cnt[row['tag']] += 1
    for tag in tag_step:
        if tag_step_cnt[tag] !=0:
            tag_step[tag] = tag_step[tag]/tag_step_cnt[tag]
    return tag_step

Overwriting mean_f.py


In [12]:
%%time
from mean_f import *
import multiprocessing as mp
files = [f'data/splitfiles/file{i}.csv' for i in range(8)]
if __name__ == '__main__':
    results = [mp.Pool(processes=8).apply(mean_f,args=(x,)) for x in files]

CPU times: user 75.6 ms, sys: 345 ms, total: 420 ms
Wall time: 28.8 s


5. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди `multiprocessing.queue` передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [13]:
%%file mean_f_mp.py
from csv import DictReader
from collections import Counter

def mean_f_mp(queue_files,queue_sum,queue_count):
    tag_step = {}
    tag_step_cnt = {}
    while not queue_files.empty():
        name = queue_files.get()
        with open(name) as csvfile:
            print(name)
            reader = DictReader(csvfile,delimiter = ';')
            for row in reader:
                if row['tag'] not in tag_step:
                    tag_step[row['tag']] = 0
                    tag_step_cnt[row['tag']] = 0
                tag_step[row['tag']] += int(row['n_steps'])
                tag_step_cnt[row['tag']] += 1
    queue_sum.put(tag_step)
    queue_count.put(tag_step_cnt)
    return True

def merging_dicts(big_dict):
    counter = Counter()
    [counter.update(i) for i in big_dict]
    return dict(big_dict[0])

Overwriting mean_f_mp.py


In [14]:
%%time
from multiprocessing import Process, Queue
from mean_f_mp import *
from collections import Counter

NP = 8
queue_files,queue_sum,queue_count = Queue(),Queue(),Queue()
done_sum,done_count = [],[]

files = [f'data/splitfiles/file{i}.csv' for i in range(8)]
[queue_files.put(name) for name in files]

for i in range(NP):
    Process(target=mean_f_mp, args=(queue_files, queue_sum,queue_count)).start()

for i in range(NP):
    done_sum.append(queue_sum.get())
    done_count.append(queue_count.get())

summ,count = merging_dicts(done_sum),merging_dicts(done_count)
                                           
for key in summ:
    summ[key] /= count[key]

CPU times: user 17.6 ms, sys: 58 ms, total: 75.7 ms
Wall time: 4.52 s
